In [1]:
import pandas as pd

#### Definir número de dígitos

In [2]:
pd.set_option("display.precision", 2)

#### Lectura de archivos

In [3]:
mundo= pd.read_csv('poblacionMundial.csv')

In [4]:
covMun= pd.read_csv('CovidMundo.csv')

In [5]:
mundo.set_index('Spanish', inplace=True)

In [6]:
covCol = pd.read_csv('CovidColombia.csv', usecols= ['Fecha de notificación', 
       'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad',
       'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'Fecha de muerte', 
       'Fecha diagnostico', 'Fecha recuperado','Tipo recuperación'], low_memory=False)

In [7]:
Col = pd.read_csv('Dane2018_2023.csv').drop(columns = ['2018',
       '2019','2021', '2022', '2023' ])

#### Convertir fechas a type "datetime"

In [8]:
fechas =['Fecha de notificación', 'Fecha de muerte', 'Fecha diagnostico',
       'Fecha recuperado']
for i in fechas:
    covCol[i] = pd.to_datetime(covCol[i])

#### Variables a ingresar

In [9]:
pais = 'Colombia'

departamento = 'Caldas'

municipio = 'Manizales'

variable_buscar = 'Atención'  # puede ser 'Atención' o 'Estado'

estado = 'Hospital UCI'  # Si atención:  ['Recuperado', 'Hospital', 'Hospital UCI', nan, 'Casa']
                   # Si Estado:   ['Leve', 'Asintomático', 'Fallecido', 'Moderado', 'Grave', nan]

ano = '2020'

#### Poblaciones

In [10]:
Poblacion_pais = mundo.loc[pais, ano]

In [11]:
total_casos_covid_colombia = covCol['Fecha de notificación'].count()


In [12]:
Fallecidos_pais = covCol['Fecha de muerte'].count()

#### Seleccionar totales;  igualar nombre de Series seleccionadas en las dataFrame; establecer índice y subíndice.

In [13]:
filtro = Col['Género'] == 'AMBOS SEXOS'
Col = Col.loc[filtro, :].drop(columns=['Género'])

In [14]:
Col.set_index(['Dpto', 'Municipio'], inplace=True)

In [15]:
covCol.rename(columns={"Departamento o Distrito ": "Dpto", "Ciudad de ubicación": "Municipio", "atención":"Atención", "Fecha diagnostico":"Fecha diagnóstico"}, inplace=True)

In [16]:
covCol.set_index(['Dpto', 'Municipio'], inplace = True)

#### Unir las dataFrames

In [17]:
Tasas = pd.merge(covCol, Col, left_index=True, right_index=True)

In [18]:
Tasas = Tasas.reset_index()

In [19]:
Tasas

,Dpto,Municipio,Fecha de notificación,Atención,Edad,Sexo,Tipo,Estado,País de procedencia,Fecha de muerte,Fecha diagnóstico,Fecha recuperado,Tipo recuperación,2020
0,Amazonas,Leticia,2020-04-14,Recuperado,26,M,Relacionado,Leve,NaN,NaT,2020-04-17,2020-05-19,PCR,49737
1,Amazonas,Leticia,2020-04-13,Recuperado,47,F,Importado,Leve,BRASIL,NaT,2020-04-18,2020-05-31,Tiempo,49737
2,Amazonas,Leticia,2020-04-14,Recuperado,16,F,Relacionado,Leve,NaN,NaT,2020-04-18,2020-05-29,PCR,49737
3,Amazonas,Leticia,2020-04-14,Recuperado,41,F,Importado,Leve,BRASIL,NaT,2020-04-18,2020-05-29,PCR,49737
4,Amazonas,Leticia,2020-04-16,Recuperado,54,M,Relacionado,Leve,NaN,NaT,2020-04-21,2020-04-29,PCR,49737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100185,Vaupés,Taraira,2020-07-12,Casa,42,F,Relacionado,Leve,NaN,NaT,2020-07-13,NaT,NaN,2423
100186,Vaupés,Taraira,2020-07-12,Casa,2,M,Relacionado,Leve,NaN,NaT,2020-07-13,NaT,NaN,2423
100187,Vaupés,Taraira,2020-07-12,Casa,27,M,Relacionado,Leve,NaN,NaT,2020-07-13,NaT,NaN,2423
100188,Vichada,La Primavera,2020-07-15,Hospital,78,F,En estudio,Moderado,NaN,NaT,2020-07-17,NaT,NaN,9608


####  Agrupar por Departamento o Municipio para hacer dataFrame con Total casos, Fallecidos y Recuperados

In [20]:
Casos_a_Fecha = Tasas.groupby(['Dpto', 'Municipio'])[['Fecha de notificación', 'Fecha de muerte',
        'Fecha recuperado']].count().loc[[departamento, municipio]]

In [21]:
Cuadro_Tasas = pd.merge(Casos_a_Fecha, Col, left_index=True, right_index=True)

In [22]:
Cuadro_Tasas.rename({'Fecha de notificación':'Total casos', 
                     'Fecha de muerte':'Fallecidos', 
                     'Fecha recuperado': 'Recuperados', 
                     '2020': 'Población'}, axis = 'columns', inplace=True)

##### Agregando columnas de tasas de letalidad y de mortalidad

In [23]:
Cuadro_Tasas['Tasa de letalidad por 100 habitantes'] = (Cuadro_Tasas['Fallecidos']/Cuadro_Tasas['Total casos'])*100
Cuadro_Tasas['Tasa de mortalidad por 100.000 habitantes'] = (Cuadro_Tasas['Fallecidos']/Cuadro_Tasas['Población'])*100000

In [24]:
Cuadro_Tasas

Total casos  Fallecidos  Recuperados  Población  \
Dpto   Municipio                                                      
Caldas Aguadas               25           0            1      23048   
       Anserma               10           0            1      36691   
       Pácora                 1           0            0      15324   
       Manizales            260           2          127     446160   
       Aranzazu               1           0            1      10556   
       Chinchiná             22           0           11      52267   
       Risaralda              1           0            1      10568   
       Supía                  5           0            1      29363   
       Neira                  8           0            1      21114   
       Viterbo                3           0            3      12853   
       Riosucio              10           0            5      51956   
       Palestina              1           0            0      15681   
       Marquetalia            8           0            7      13490   
       La Dorada            127           2           77      74006   
       Samaná                 2           0            2      19967   
       Manzanares            13           1            2      17794   
       Marmato               25           0            1       9167   
       Villamaría            28           2           15      67429   
       Norcasia               1           1            0       6128   

                    Tasa de letalidad por 100 habitantes  \
Dpto   Municipio                                           
Caldas Aguadas                                      0.00   
       Anserma                                      0.00   
       Pácora                                       0.00   
       Manizales                                    0.77   
       Aranzazu                                     0.00   
       Chinchiná                                    0.00   
       Risaralda                                    0.00   
       Supía                                        0.00   
       Neira                                        0.00   
       Viterbo                                      0.00   
       Riosucio                                     0.00   
       Palestina                                    0.00   
       Marquetalia                                  0.00   
       La Dorada                                    1.57   
       Samaná                                       0.00   
       Manzanares                                   7.69   
       Marmato                                      0.00   
       Villamaría                                   7.14   
       Norcasia                                   100.00   

                    Tasa de mortalidad por 100.000 habitantes  
Dpto   Municipio                                               
Caldas Aguadas                                           0.00  
       Anserma                                           0.00  
       Pácora                                            0.00  
       Manizales                                         0.45  
       Aranzazu                                          0.00  
       Chinchiná                                         0.00  
       Risaralda                                         0.00  
       Supía                                             0.00  
       Neira                                             0.00  
       Viterbo                                           0.00  
       Riosucio                                          0.00  
       Palestina                                         0.00  
       Marquetalia                                       0.00  
       La Dorada                                         2.70  
       Samaná                                            0.00  
       Manzanares                                        5.62  
       Marmato                                           0.00  
       Villamaría                                  

#### Condiciones de búsqueda

In [25]:
if departamento == '' and municipio== '':
    print('Debe ingresar mínimo el Departamento.  Inténtelo nuevamente')
    pass
else:
    if municipio == '':
        Total_casos = Cuadro_Tasas.loc[departamento, 'Total casos'].astype(int) 
        Fallecidos_dpto_munic = Cuadro_Tasas.loc[(departamento), 'Fallecidos'].astype(int)
        Poblacion_dpto_munic = Cuadro_Tasas.loc[(departamento), 'Población'].astype(int)
    
    else:
        Total_casos = Cuadro_Tasas.loc[(departamento, municipio), 'Total casos'].astype(int)
        Fallecidos_dpto_munic = Cuadro_Tasas.loc[(departamento, municipio), 'Fallecidos'].astype(int)
        Poblacion_dpto_munic = Cuadro_Tasas.loc[(departamento, municipio), 'Población'].astype(int)
    

In [26]:
total_casos_covid_colombia

226373

In [27]:
Fallecidos_dpto_munic    

2

In [28]:
Fallecidos_pais

7961

In [29]:
Poblacion_dpto_munic

446160

In [30]:
Poblacion_dpto_munic.sum()

446160

In [31]:
Poblacion_pais

50883000

In [32]:
Total_casos

260

In [33]:
Total_casos_dpto = Total_casos.sum()

Total_casos_dpto

260

#### Aplicar filtro propuesto en variable "estado" por Departamento o Municipio

In [35]:
hoy = pd.Timestamp.today()


In [36]:
filtro= Tasas[variable_buscar]== estado
if municipio == '':
    Cuadro_filtrados = Tasas.loc[filtro, 'Atención': 'Tipo recuperación'].loc[(Tasas['Dpto'] == departamento)]
    numero_estados= Cuadro_filtrados.loc[:, variable_buscar].count()
    print('Según datos reportados al ', hoy, ' por el INS Colombiano, de los ' + str(Total_casos_dpto) 
          + ' casos de '+ departamento + ', ' + str(numero_estados) +' casos, es decir el '
          + str(((numero_estados/Total_casos_dpto)*100).round(2))+' %, está en condición: '
          + variable_buscar + ' ' + estado + ' y representan el ' 
          + str(((numero_estados/Poblacion_dpto_munic.sum())*100).round(6)) + ' %  de los ' 
          + str(Poblacion_dpto_munic.sum()) + ' habitantes de ' + departamento + '.','\n')
   
    print('La tasa de mortalidad en los municipios del Departamento de ' 
        + departamento + ' por cien mil habitantes es: ' + str(((Fallecidos_dpto_munic/Poblacion_dpto_munic)*100000).round(2)),'\n')
    
    print('La tasa de letalidad en los municipios del Departamento de ' + departamento + 
          ' por cien enfermos de CoVid-19, es: ' + str(((Fallecidos_dpto_munic/Total_casos)*100).round(2)),'\n')

else:
    Cuadro_filtrados = Tasas.loc[filtro, 'Atención': 'Tipo recuperación'].loc[(Tasas['Dpto'] == departamento) & (Tasas['Municipio']== municipio)]
    numero_estados= Cuadro_filtrados.loc[:, variable_buscar].count()
    print('Según datos reportados al', hoy, 'por el INS Colombiano, de los ' + str(Total_casos) + ' casos de '+ municipio + ', ' 
          + str(numero_estados) +' casos, es decir el '+ str(((numero_estados/Total_casos)*100).round(2))
          +' %, están en '+ variable_buscar + ': ' + estado + ' y representan el '
          + str(((numero_estados/Poblacion_dpto_munic.sum())*100).round(6)) 
          + ' % de los ' + str(Poblacion_dpto_munic.sum()) + ' habitantes de ' + municipio +'.','\n')
    
    print('La tasa de letalidad en el municipio de '+ municipio + ',' + ' Departamento de ' 
        + departamento + ' es de ' + str(((Fallecidos_dpto_munic/Total_casos)*100).round(2)) 
        + ' por cien enfermos de CoVid-19.','\n')
    
    print('La tasa de mortalidad en el municipio de '+ municipio + ',' + ' Departamento de ' 
        + departamento + ' es de ' + str(((Fallecidos_dpto_munic/Poblacion_dpto_munic)*100000).round(2)) 
        + ' por cien mil habitantes.','\n')
    
print('La tasa de mortalidad de ' + pais+ ' por CoV-2 ' + 'es de ' + str(((Fallecidos_pais/Poblacion_pais)*1000000).round(1)) 
      + ' por millón de habitantes.','\n')

print('La tasa de letalidad de '+pais+ ' por CoV-2 ' + 'es de ' + str(((Fallecidos_pais/total_casos_covid_colombia)*100).round(2)) 
      + ' por cien enfermos de CoVid-19.','\n')

Según datos reportados al 2020-07-24 16:52:50.479185 por el INS Colombiano, de los 260 casos de Manizales, 3 casos, es decir el 1.15 %, están en Atención: Hospital UCI y representan el 0.000672 % de los 446160 habitantes de Manizales. 

La tasa de letalidad en el municipio de Manizales, Departamento de Caldas es de 0.77 por cien enfermos de CoVid-19. 

La tasa de mortalidad en el municipio de Manizales, Departamento de Caldas es de 0.45 por cien mil habitantes. 

La tasa de mortalidad de Colombia por CoV-2 es de 156.5 por millón de habitantes. 

La tasa de letalidad de Colombia por CoV-2 es de 3.52 por cien enfermos de CoVid-19. 

